In [2]:
# Data
# https://www.kaggle.com/siddarthareddyt/cats-and-dogs/data

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

%matplotlib inline

/Users/administrator/anaconda3/envs/py37/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train_path='/Users/administrator/Research/trainings/learnings/data/cats-and-dogs/train'
valid_path='/Users/administrator/Research/trainings/learnings/data/cats-and-dogs/valid'
test_path='/Users/administrator/Research/trainings/learnings/data/cats-and-dogs/test'

In [15]:
train_batches=ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=['dogs','cats'],batch_size=20)
valid_batches=ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),classes=['dogs','cats'],batch_size=20)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [19]:

def plots(ims,figsize=(12,6),rows=2,interp=False,titles=None):
    if type(ims[0]) is np.ndarray:
        ims=np.array(ims).astype(np.uint8)
        if(ims.shape[-1]!=3):
            ims=ims.transpose((0,2,3,1))
    f=plt.figure(figsize=(100,100))
    cols=len(ims)//rows if len(ims)%2==0 else len(ims)//rows+1
    for i in range(len(ims)):
        sp=f.add_subplot(rows,cols,i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i],fontsize=16)
        plt.imshow(ims[i],interpolation=none if interp else 'none')

In [20]:
imgs,labels=next(train_batches)

In [21]:
plots(imgs,titles=labels)
# Dogs: [1,0]
# Cats[0,1]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [9]:
# Shape of the batch of images
#for img in imgs:
#    print(img.shape)

In [10]:
model = Sequential(
[
    Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
    Flatten(),
    Dense(2,activation='softmax')
]
)

In [11]:
model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit_generator(train_batches,steps_per_epoch=20,validation_data=valid_batches,validation_steps=50,epochs=5,verbose=1)

Epoch 1/5
20/20 [==============================] - 25s 1s/step - loss: 8.5829 - acc: 0.4675 - val_loss: 8.1235 - val_acc: 0.4960
Epoch 2/5
20/20 [==============================] - 23s 1s/step - loss: 8.5829 - acc: 0.4675 - val_loss: 7.8817 - val_acc: 0.5110
Epoch 3/5
20/20 [==============================] - 23s 1s/step - loss: 7.5352 - acc: 0.5325 - val_loss: 8.2525 - val_acc: 0.4880
Epoch 4/5
20/20 [==============================] - 24s 1s/step - loss: 8.3411 - acc: 0.4825 - val_loss: 7.7044 - val_acc: 0.5220
Epoch 5/5
20/20 [==============================] - 24s 1s/step - loss: 8.3814 - acc: 0.4800 - val_loss: 8.3331 - val_acc: 0.4830


# Fine tuning the existing model : vgg16

In [23]:
vgg16_model=keras.applications.vgg16.VGG16()

553467904/553467096 [==============================] - 76s 0us/step


In [24]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [25]:
type(vgg16_model)

keras.engine.training.Model

In [26]:
model=Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [29]:
model.layers.pop()

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [33]:
for layer in model.layers:
    layer.trainable = False

In [34]:
model.add(Dense(2,activation='softmax'))

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [36]:
model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches,steps_per_epoch=20,validation_data=valid_batches,validation_steps=50,epochs=5,verbose=1)

Epoch 1/5
 1/20 [>.............................] - ETA: 3:08 - loss: 0.6852 - acc: 0.6500